In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
# for reproducibility
np.random.seed(1671)

# network and training
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

In [3]:
# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [4]:
#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#one-hot representation of the labels
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# Baseline model with single layer

In [9]:
# Build the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(NB_CLASSES, input_shape=(RESHAPED, ), name='dense_layer', activation='softmax'))
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer (Dense)         (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
# Training the model.
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Epoch 1/200
375/375 [==============================] - 1s 2ms/step - loss: 0.7413 - accuracy: 0.8208 - val_loss: 0.4064 - val_accuracy: 0.8937
Epoch 2/200
375/375 [==============================] - 0s 1ms/step - loss: 0.3875 - accuracy: 0.8967 - val_loss: 0.3317 - val_accuracy: 0.9099
Epoch 3/200
375/375 [==============================] - 0s 1ms/step - loss: 0.3370 - accuracy: 0.9081 - val_loss: 0.3066 - val_accuracy: 0.9143
Epoch 4/200
375/375 [==============================] - 0s 1ms/step - loss: 0.3141 - accuracy: 0.9129 - val_loss: 0.2917 - val_accuracy: 0.9192
Epoch 5/200
375/375 [==============================] - 0s 1ms/step - loss: 0.3001 - accuracy: 0.9160 - val_loss: 0.2848 - val_accuracy: 0.9208
Epoch 6/200
375/375 [==============================] - 0s 1ms/step - loss: 0.2907 - accuracy: 0.9192 - val_loss: 0.2775 - val_accuracy: 0.9226
Epoch 7/200
375/375 [==============================] - 0s 1ms/step - loss: 0.2836 - accuracy: 0.9204 - val_loss: 0.2747 - val_accuracy: 0.9234

In [12]:
# evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 0s 597us/step - loss: 0.3013 - accuracy: 0.9256
Test accuracy: 0.925599992275238


In [13]:
# get predictions
print(np.argmax(model.predict(X_test),axis=1))

[7 2 1 ... 4 5 6]


In [14]:
# save and clear tf session
model.save('saved_model/model_baseline')
del model
tf.keras.backend.clear_session()

INFO:tensorflow:Assets written to: saved_model/model_baseline/assets


# Adding two hidden layers to the model

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED, ), name='dense_layer_1', activation='relu'),
    tf.keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'),
    tf.keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax')
])
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_1 (Dense)       (None, 128)               100480    
                                                                 
 dense_layer_2 (Dense)       (None, 128)               16512     
                                                                 
 dense_layer_3 (Dense)       (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
# Training the model.
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Epoch 1/200
375/375 [==============================] - 1s 3ms/step - loss: 0.3570 - accuracy: 0.8990 - val_loss: 0.1805 - val_accuracy: 0.9463
Epoch 2/200
375/375 [==============================] - 1s 2ms/step - loss: 0.1478 - accuracy: 0.9563 - val_loss: 0.1261 - val_accuracy: 0.9634
Epoch 3/200
375/375 [==============================] - 1s 2ms/step - loss: 0.1006 - accuracy: 0.9695 - val_loss: 0.1046 - val_accuracy: 0.9693
Epoch 4/200
375/375 [==============================] - 1s 2ms/step - loss: 0.0742 - accuracy: 0.9777 - val_loss: 0.1046 - val_accuracy: 0.9697
Epoch 5/200
375/375 [==============================] - 1s 2ms/step - loss: 0.0561 - accuracy: 0.9829 - val_loss: 0.0933 - val_accuracy: 0.9711
Epoch 6/200
375/375 [==============================] - 1s 2ms/step - loss: 0.0443 - accuracy: 0.9859 - val_loss: 0.0926 - val_accuracy: 0.9737
Epoch 7/200
375/375 [==============================] - 1s 3ms/step - loss: 0.0360 - accuracy: 0.9889 - val_loss: 0.0869 - val_accuracy: 0.9747

In [16]:
# evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 0s 891us/step - loss: 0.2258 - accuracy: 0.9823
Test accuracy: 0.9822999835014343


In [17]:
model.save('saved_model/model_v1')
del model
tf.keras.backend.clear_session()

INFO:tensorflow:Assets written to: saved_model/model_v1/assets
